In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed
from keras.layers import LSTM, SimpleRNN, GRU
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
biblia=open("biblia.txt","r", encoding = "ISO-8859-1").read().lower().replace("+","")
print('corpus length:', len(biblia))

corpus length: 3855184


In [3]:
chars = sorted(list(set(biblia)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(biblia) - maxlen, step):
    sentences.append(biblia[i: i + maxlen])
    next_chars.append(biblia[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 1285048


In [5]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [16]:
print('Build model...')
model = Sequential()
model.add(GRU(128, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(50)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(200))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(len(chars)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('softmax'))

Build model...


In [18]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [19]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
gru_3 (GRU)                      (None, 40, 128)       71424       gru_input_3[0][0]                
____________________________________________________________________________________________________
activation_6 (Activation)        (None, 40, 128)       0           gru_3[0][0]                      
____________________________________________________________________________________________________
batchnormalization_9 (BatchNorma (None, 40, 128)       512         activation_6[0][0]               
____________________________________________________________________________________________________
timedistributed_3 (TimeDistribut (None, 40, 50)        6450        batchnormalization_9[0][0]       
___________________________________________________________________________________________

In [21]:
model.fit(X, y, batch_size=2048, nb_epoch=1)

Epoch 1/1
1285048/1285048 [==============================] - 3836s - loss: 2.8340     

In [26]:
model.optimizer.lr.assign(0.003)

<tf.Tensor 'Assign_146:0' shape=() dtype=float32_ref>

In [ ]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=2048, nb_epoch=1)

    start_index = random.randint(0, len(biblia) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = biblia[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
        print(generated)


--------------------------------------------------
Iteration 1
Epoch 1/1
1285048/1285048 [==============================] - 3524s - loss: 2.5979     

In [ ]:
model.optimizer.lr.assign(0.001)

## With embedding

In [12]:
X_new=[]
for sentence in sentences:
    X_new.append([char_indices[char] for char in sentence])

In [29]:
model_e = Sequential()
model_e.add(Embedding(len(chars),50, input_length=maxlen))
model_e.add(BatchNormalization())
model_e.add(LSTM(256, consume_less='gpu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(200, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(len(chars),activation='softmax'))

In [30]:
model_e.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model_e.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_9 (Embedding)          (None, 40, 50)        2850        embedding_input_9[0][0]          
____________________________________________________________________________________________________
batchnormalization_19 (BatchNorma(None, 40, 50)        100         embedding_9[0][0]                
____________________________________________________________________________________________________
lstm_12 (LSTM)                   (None, 256)           314368      batchnormalization_19[0][0]      
____________________________________________________________________________________________________
batchnormalization_20 (BatchNorma(None, 256)           512         lstm_12[0][0]                    
___________________________________________________________________________________________

In [ ]:
model_e.fit(X_new, y, batch_size=4096, nb_epoch=1)

Epoch 1/1
